##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#Google Drive Mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#Import Package
from __future__ import absolute_import, division, print_function, unicode_literals
#import tensorflow 
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense,Reshape,Flatten,Conv2D,Conv2DTranspose,LeakyReLU,ReLU,GlobalAveragePooling1D
from tensorflow.keras.layers import BatchNormalization,Dropout,Embedding,Activation,Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
#from tensorflow.keras.preprocessing.image import image
from keras.utils.np_utils import to_categorical 
from tensorflow.keras import backend as K
K.clear_session()
#other library 
import os
import numpy as np
import matplotlib.pyplot as plt
import io
from PIL import Image
import math
import datetime
import pandas as pd
import random
from tensorboard.plugins.hparams import api as hp
#installation
!pip install tensorflow-addons==0.8.3
#import tensorflow_addons as tfa
from mlxtend.plotting import plot_confusion_matrix

In [ ]:
#Dataset
import zipfile
zip_ref = zipfile.ZipFile(" ", 'r') #Dataset File Path
zip_ref.extractall("/content/")
zip_ref.close()

In [ ]:
#Data Preprocessing with augmentation

TRAINING_DIR = " " #Training Set Directory
training_datagen = ImageDataGenerator(rescale = 1./255,)
                                     
VALIDATION_DIR = " " #Validation Set Directory
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(78,78),
    class_mode='categorical',
    batch_size=64
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(78,78),
    class_mode='categorical',
    batch_size=64
)

In [ ]:
#Model 1
input = Input(shape=(84,84,3))
pre_trained_model = tf.keras.applications.inception_v3.InceptionV3(input_shape = (84, 84, 3), 
                                input_tensor = input,
                                include_top = False,
                                weights = None 
                                )
for layer in pre_trained_model.layers:
  layer.trainable = True
inception_layer = pre_trained_model.get_layer('mixed10')
inception_layer = inception_layer.output
inception_layer = layers.Flatten()(inception_layer)
print('last layer output shape: ', inception_layer.shape)

In [ ]:
#Model 2
pre_trained_model_1 = tf.keras.applications.resnet50.ResNet50(input_shape = (84, 84, 3), 
                                input_tensor = input,
                                include_top = False, 
                                weights = None 
                                )
for layer in pre_trained_model_1.layers:
  layer.trainable = True
resnet_layer = pre_trained_model_1.get_layer('conv5_block3_out')
#pre_trained_model_1.summary()
resnet_layer  = resnet_layer .output
#x2 = layers.Conv2D(2048,(4,4),activation='relu')(x2)
resnet_layer  = layers.Flatten()(resnet_layer )
print('last layer output shape: ', resnet_layer.shape)

In [ ]:
#Vision Transformer
#Tuneble Prameters
weight_decay = 0.0001
num_epochs = 100
num_classes=4
image_size = 78 
patch_size = 6   
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
] 
transformer_layers = 8
mlp_head_units = [2048, 1024]

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
patches = Patches(patch_size)(input)
encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

for _ in range(transformer_layers):
  x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
  attention_output = layers.MultiHeadAttention(
  num_heads=num_heads, key_dim=projection_dim, dropout=0.1
  )(x1, x1)
  x2 = layers.Add()([attention_output, encoded_patches])
  x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
  x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
  encoded_patches = layers.Add()([x3, x2])
representation = layers.LayerNormalization(name='last_layer',epsilon=1e-6)(encoded_patches)
representation = layers.Flatten()(representation)
representation = layers.Dropout(0.5)(representation)
representation= mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
print('last layer output shape: ', representation.shape)

In [ ]:
#Merge Model
last_layer = layers.Concatenate()([inception_layer , resnet_layer, representation])
x = layers.Dense(1024, activation='relu')(last_layer)
x = tf.keras.layers.Dropout(0.2)(x)                  
x = layers.BatchNormalization(name='batch_1')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = ReLU()(x)
x = Dense(512)(x)
x = layers.BatchNormalization(name='batch_2')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = ReLU()(x)
x = Dense(256)(x)
x = BatchNormalization(name='batch_3')(x)
x = ReLU()(x)
x = layers.Dense  (4, activation='softmax')(x)

In [ ]:
#Callback
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor= ,
                              patience= , min_lr= ,verbose=1) #Tuneable Parameter

In [ ]:
#Model Compile 
model = Model(inputs=input,outputs= x)    
model.compile(optimizer = Adam(learning_rate=0.001),
              loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False), 
              metrics = ['accuracy','Precision', 'Recall'])

In [ ]:
#Model 
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 20,
            verbose = 1,
            callbacks=[reduce_lr],
            )

In [ ]:
#Test Dataset
TEST_DIR = " " #Test Set Filepath
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
	TEST_DIR,
	target_size=(78,78),
	class_mode='categorical',
	shuffle=False,
  batch_size=64
)

In [ ]:
#Model Evaluation
results = model.evaluate(test_generator)

In [ ]:
#Terminate
import os, signal
os.kill(os.getpid(), signal.SIGKILL)